In [163]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [164]:
# import codecs
# df = codecs.open('../wellcome.csv', "rb", encoding='latin-1') 

# for encoding (if needed for futher writes): df['Journal title'].str.encode('utf-8');

In [165]:
df = pd.read_csv("../wellcome.csv", encoding ='latin-1')
df.head(5)

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


## Data Inconsistencies: 
- Mix of PubMed ID and PMC ID. Observations with both IDs.
- Publisher name  in different ways.  For example, OUP, Oxford University Press, and O.U.P.
- Journal name. For example, PNAS, Proceedings of the National Academy of Sciences, P.N.A.S.
- Different prices cited for the same journal 
    - When these differences are small, it can be assumed that this reflects fluctuations in currency prices.  
    - Where they are significant, then it should be assumed that other funders have contributed to the cost of that APC, and the figure cited is the Wellcome contribution to that APC.

In [166]:
df['Journal title'].describe()

count         2126
unique         984
top       PLoS One
freq            92
Name: Journal title, dtype: object

In [167]:
# convert to lower case
df['Journal title'] = df['Journal title'].str.lower()

In [168]:
# convert & to and
df['Journal title'] = df['Journal title'].str.replace('&', 'and')

In [169]:
# convert J, Jnl, Jounral, Jounal, Journals to journal
regex_journal = re.compile('j(([a-z])*)?')
df['Journal title'] = df['Journal title'].str.replace(regex_journal, 'journal')

In [170]:
# delete the
df['Journal title'] = df['Journal title'].str.replace('the', '')
# delete dots
df['Journal title'] = df['Journal title'].str.replace('.', '')

In [171]:
# remove sections of a journal following colon, comma, dash
regex_colon = re.compile(':  ?.*')
regex_comma = re.compile(',  ?.*')
regex_dash = re.compile('-  ?.*')
df['Journal title'] = df['Journal title'].str.replace(regex_colon, '')
df['Journal title'] = df['Journal title'].str.replace(regex_comma, '')
df['Journal title'] = df['Journal title'].str.replace(regex_dash, '')

In [172]:
# Sci to Science = str
df['Journal title'].replace(to_replace='(sci)$', value='science', inplace=True, regex=True)

In [173]:
# Biol to Biological str
df['Journal title'].replace(to_replace='(biol )', value='biological ', inplace=True, regex=True)
df['Journal title'].replace(to_replace='(biol)$', value='biological', inplace=True, regex=True);

In [174]:
# space at the end of the sentence
df['Journal title'] = df['Journal title'].str.strip()

In [175]:
# Regex replace: str
    # biochimica et bioohysica acta.* to bba
df['Journal title'].replace(to_replace='.*acta.*', value='bba', inplace=True, regex=True)

# Fromatting replace
    # '  ' to ' '
df['Journal title'] = df['Journal title'].str.replace('  ', ' ')

### Other text iterations
#### Replace of 1:1 cases:
    # Epidemiol = Epidemiology
    # US English vs British English: Behavioural and Behavioral
    # BJOPHTHALMOL = British Journal of Ophthalmology
    # Opthalmology = Ophthalmology
    # Sex Transm Infect. = Sexually Transmitted Infections 
    # 'American Journal for Clinical Nutrition' = 'American Journal of Clinical Nutrition'
    # Am to American
    # molecluar to molecular

#### Regex replace: 
    # Proceedings of the National Academy of Sciences of.* to PNAS
    # Chem to Chemistry
    # Int, Intl to International

In [176]:
# test
df['Journal title'].sort_values().unique();
df['Journal title'].describe()

count         2126
unique         823
top       plos one
freq           191
Name: Journal title, dtype: object

## Task questions:

### Five most common journals and the total articles for each

In [177]:
df['Journal title'].value_counts().nlargest(5)

plos one                           191
journal of biological chemistry     60
neuroimage                          34
nucleic acids research              26
plos pathogens                      24
Name: Journal title, dtype: int64

### Mean, median, and standard deviation of the open-access cost per article for each journal 

In [178]:
costs = 'COST (£) charged to Wellcome (inc VAT when charged)'
# df[costs].describe();
df[costs] = df[costs].str.replace('£','')
df[costs] = df[costs].str.replace('$','') 

# How to replace a list of values?

#### (!) Converting / Seeing the unconvertable data types

In [179]:
df[costs] = pd.to_numeric(df[costs]);

In [180]:
group = df.groupby('Journal title').agg({costs: ['std', 'mean', 'median']});

# Using ravel, and a string join, we can create 1 level names for the columns:
group.columns = [" ".join(x) for x in group.columns.ravel()];

In [181]:
group.head(5)

,COST (£) charged to Wellcome (inc VAT when charged) std,COST (£) charged to Wellcome (inc VAT when charged) mean,COST (£) charged to Wellcome (inc VAT when charged) median
Journal title,,,
academy of nutrition and dietetics,NaN,2379.540,2379.540
acs chemical biology,507.309560,1418.186,1294.590
acs chemical neuroscience,NaN,1186.800,1186.800
acs nano,35.708892,668.140,668.140
addiction,306.481292,2136.225,2136.225
